# IMPORTING THE REQ LIBRARIES

In [4]:
from langchain_community.document_loaders import UnstructuredURLLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_openai import OpenAI
from dotenv import load_dotenv

# LOADING OF THE DATA

In [5]:
urls = ['https://www.victoriaonmove.com.au/local-removalists.html','https://www.victoriaonmove.com.au/contact.html','https://www.victoriaonmove.com.au/melbourne-interstate-removalists.html']
loader= UnstructuredURLLoader(urls=urls)
data = loader.load()

In [6]:
data

[Document(metadata={'source': 'https://www.victoriaonmove.com.au/local-removalists.html'}, page_content='Loading...\n\nLOCAL REMOVALS\n\nYour trusted partner in seamless moving and packing solutions!\n\nGoogle logo G\n\nGoogle Rating\n\nGold star icon\n\nGold star icon\n\nGold star icon\n\nGold star icon\n\nGold star icon\n\n5 stars, 111 reviews\n\n\n\nRequst A call for You:\n\nLocal removal services via "Victoria on move"\n\nVictoria on Move is your trusted local moving company in Melbourne, specializing in seamless relocation services. As experienced furniture movers and relocation experts, we provide top-notch packing and moving services tailored to your needs. Whether you\'re moving across town or relocating interstate, our professional movers ensure a stress-free experience. Count on Victoria on Move for reliable removal services, making us the preferred choice among local movers in Melbourne. Discover why we\'re recognized for our commitment to quality and customer satisfaction.\

# SPLITTING OF TEXT 

In [7]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)
print("THE TOTAL NUMBER OF DOCUMENTS",len(docs))

THE TOTAL NUMBER OF DOCUMENTS 17


# INITIALIZING THE VECTOR DATABASE - CHROMA DB

In [8]:
load_dotenv()

True

In [ ]:
vectorstore= Chroma.from_documents(documents=docs,embedding=OpenAIEmbeddings())   # defining the vector database and the embedding model
retriever=vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":3}) # defining the search method and number of docs to be retrieved
retrieved_docs=retriever.invoke("What are the services provided by Victoria On Move?") # defining the query
len(retrieved_docs)

3

In [29]:
print(retrieved_docs[0].page_content)

Transparent Pricing:

Clear and competitive pricing with no hidden fees, providing transparency and peace of mind.

Positive Reputation:

Established reputation for reliability, professionalism, and positive customer feedback in the moving industry.

Choosing Victoria on Move ensures a stress-free and reliable moving experience, backed by expertise, professionalism, and a commitment to customer satisfaction.

Find removalists travelling interstate to or from Melbourne

From Melbourne

To Melbourne


# CONNECTING THE OUTPUT TO THE LLM TO GET THE MOST ACCURATE ANS

In [14]:
! pip install faiss-cpu beautifulsoup4

   ---------------------------------------- 0.0/18.2 MB ? eta -:--:--
   -------- ------------------------------- 3.9/18.2 MB 21.3 MB/s eta 0:00:01
   ------------------- -------------------- 8.7/18.2 MB 21.5 MB/s eta 0:00:01
   ------------------------------ --------- 13.9/18.2 MB 22.9 MB/s eta 0:00:01
   ---------------------------------------- 18.2/18.2 MB 22.5 MB/s  0:00:00


In [ ]:
from langchain_community.vectorstores import FAISS

In [28]:
llm=OpenAI(temperature=0.7,max_tokens=500)
response=llm.invoke(f"Answer the question based on the context provided: {retrieved_docs[0].page_content} \n Question: What are the services provided by Victoria On Move?")
print(response)

 

 Answer: Victoria On Move provides clear and competitive pricing with no hidden fees, transparency and peace of mind, a positive reputation for reliability and professionalism, and a stress-free and reliable moving experience backed by expertise, professionalism, and a commitment to customer satisfaction. They also offer removalist services for both interstate moves to and from Melbourne.
